In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
!pip install natsort
!pip install tensorflow_datasets

In [ ]:
#imports
from zipfile import ZipFile
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import io
import numpy as np
import random
import cv2
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from pathlib import Path
from natsort import natsorted
import sklearn

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3


from sklearn.model_selection import train_test_split

#from preprocessing import Preprocessing

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# auxiliary function that you are already familiar with
import zipfile
def get_zip(file_name):
    '''
    file_name = Name of zip file you want to download from object storage
    '''
    try:
        fobj = open(file_name, "wb")
        fobj.write(project.get_file(file_name).read()) 
        fobj.close()
        z = zipfile.ZipFile(file_name)
        z.extractall()
    except Exception as e:
        print(Exception,e)
    else:
        print('Files downloaded successfully')

In [ ]:
get_zip('preprocessed.zip')

In [ ]:
metadata_csv = cos_client.get_object(Bucket=bucket, Key='cancer_metadata.csv')['Body']
metadata = pd.read_csv(metadata_csv)
metadata.columns

In [ ]:
benign_count = len(metadata[ metadata['benign_malignant'] == 'benign'])
malignant_count = len(metadata[ metadata['benign_malignant'] == 'malignant'])
malignant_frac = malignant_count / (benign_count + malignant_count) * 100

print(benign_count)
print(malignant_count)
print(malignant_frac)

In [ ]:
#stratified sampling
metadata = metadata.iloc[range(int(len(metadata)* 0.5))]
f = 0.35
train_df, test_df = train_test_split(metadata, test_size=f)
train_df, val_df = train_test_split(train_df, test_size=f)

# train_df = metadata.groupby('benign_malignant', group_keys=False).apply(lambda x: x.sample(frac=f))

benign_count = len(train_df[ train_df['benign_malignant'] == 'benign'])
malignant_count = len(train_df[ train_df['benign_malignant'] == 'malignant'])
malignant_frac = malignant_count / (benign_count + malignant_count) * 100

print(f"Benign: {benign_count}")
print(f"Malignant: {malignant_count}")
print(f"Malignant fraction: {malignant_frac:.2f}%")

In [ ]:
benign_count = len(val_df[ val_df['benign_malignant'] == 'benign'])
malignant_count = len(val_df[ val_df['benign_malignant'] == 'malignant'])
malignant_frac = malignant_count / (benign_count + malignant_count) * 100

print(f"Benign: {benign_count}")
print(f"Malignant: {malignant_count}")
print(f"Malignant fraction: {malignant_frac:.2f}%")

In [ ]:
benign_count = len(test_df[ test_df['benign_malignant'] == 'benign'])
malignant_count = len(test_df[ test_df['benign_malignant'] == 'malignant'])
malignant_frac = malignant_count / (benign_count + malignant_count) * 100

print(f"Benign: {benign_count}")
print(f"Malignant: {malignant_count}")
print(f"Malignant fraction: {malignant_frac:.2f}%")

In [ ]:
train_df

In [ ]:
pos_features = train_df[ train_df['benign_malignant'] == 'malignant'].reset_index()
neg_features = train_df[ train_df['benign_malignant'] == 'benign'].reset_index()

ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features.iloc[choices]
res_df = pd.concat([res_pos_features, neg_features])

neg = len(neg_features)
print(neg_features.shape)
print(pos_features.shape)
print(res_pos_features.shape)
print(res_df.shape)

In [ ]:
import shutil
import os

def local_filename(row):
    return row['isic_id'] + ".JPG"

def download_df(df, folder="cancer"):

    def save_image(img_name, path_prefix):
        src_filepath = os.path.join(source_folder, img_name)
        dest_filepath = os.path.join(path_prefix, img_name)
        shutil.copyfile(src_filepath, dest_filepath)

    benin = df[df['benign_malignant'] == 'benign']
    malignat = df[df['benign_malignant'] == 'malignant']

    benin_filenames = benin.apply(local_filename, axis=1)
    malignat_filenames = malignat.apply(local_filename, axis=1)

    p = os.path.join(folder, "")
    os.makedirs(p, exist_ok=True)
    benin_path = os.path.join(p, 'benign')
    os.makedirs(benin_path, exist_ok=True)
    malignat_path = os.path.join(p, 'malignant')
    os.makedirs(malignat_path, exist_ok=True)

    source_folder = "preprocessed"  # Specify the path to your local folder here

    for img_name in benin_filenames:
        save_image(img_name, benin_path)

    for img_name in malignat_filenames:
        save_image(img_name, malignat_path)


In [ ]:
download_df(train_df, "cancer/train")

In [ ]:
download_df(val_df, "cancer/val")

In [ ]:
download_df(test_df, "cancer/test")

In [ ]:
def copy_resampled_images(resampled_df, path):
    pos_features =  resampled_df[ resampled_df['benign_malignant'] == 'malignant']
    img_names = pos_features.apply(bucket_filename, axis=1)
    
    p = Path(path)
    
    for (img_name, count) in img_names.groupby(img_names).count().items():
        orig_img = p / img_name
        for i in range(count-1):
            copied_name = str(orig_img) + "_" + str(i)
            !cp {orig_img} {copied_name}

In [ ]:
img_height, img_width = 100, 100
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    "cancer/train",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary')

val_ds = tf.keras.utils.image_dataset_from_directory(
    "cancer/val",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary')

test_ds = tf.keras.utils.image_dataset_from_directory(
    "cancer/test",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary')

In [ ]:
class_names = train_ds.class_names
#print(class_names)
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[int(labels[i] > 0.5)])
    plt.axis("off")

In [ ]:
from xgboost import XGBClassifier

train_ds = train_ds.unbatch()
X_train = np.array(list(train_ds.map(lambda x, y: x)))
y_train = np.array(list(train_ds.map(lambda x, y: y)))

# Reshape the input data
num_samples, height, width, channels = X_train.shape
X_train = X_train.reshape(num_samples, height * width * channels)

In [ ]:
val_ds = val_ds.unbatch()
X_val = np.array(list(val_ds.map(lambda x, y: x)))
y_val = np.array(list(val_ds.map(lambda x, y: y)))

num_samples, height, width, channels = X_val.shape
X_val = X_val.reshape(num_samples, height * width * channels)

In [ ]:
test_ds = test_ds.unbatch()
X_test = np.array(list(test_ds.map(lambda x, y: x)))
y_test= np.array(list(test_ds.map(lambda x, y: y)))

num_samples, height, width, channels = X_test.shape
X_test = X_test.reshape(num_samples, height * width * channels)

In [ ]:
neg = len(metadata[metadata['benign_malignant'] == 'benign'])
pos = len(metadata[metadata['benign_malignant'] == 'malignant'])
total = neg + pos
initial_bias = np.log([pos / neg])

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

print(f"weight for 0: {weight_for_0}")
print(f"weight for 1: {weight_for_1}")

class_weight = {0: weight_for_0, 1: weight_for_1}

In [ ]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'])
        for candidate in candidates:
            print('Model with rank: {0}'.format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate], 
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
xgb_model = XGBClassifier(scale_pos_weight=class_weight[1], n_iter = 100, random_state = 42)

params = {
    "gamma": [i/100 for i in range(0, 51, 10)],
    "learning_rate": [i/100 for i in range(1, 26, 6)],
    "max_depth": range(2, 10, 2),
    "reg_alpha": range(1, 100, 25),
}

In [ ]:
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, RandomizedSearchCV

grid_search = GridSearchCV(
                            estimator = xgb_model,
                            param_grid=params,
                            scoring='roc_auc',
                            n_jobs = 1,
                            cv = 3,
                            verbose=True)
grid_search.fit(X_train, y_train)
report_best_scores(search.cv_results_, 3)

In [ ]:
#y_pred = XGB.predict(X_test)

In [ ]:
#from sklearn import metrics
#print(f"f1 = {metrics.f1_score(y_test, y_pred)}")
#print(f"precision = {metrics.precision_score(y_test, y_pred)}")
#print(f"recall = {metrics.recall_score(y_test, y_pred)}")
#print(f"accuracy = {metrics.accuracy_score(y_test, y_pred)}")
#print(f"MCC = {metrics.matthews_corrcoef(y_test, y_pred)}")


In [ ]:
#import pickle

#filename = 'xgb_model_smaller_19_06.pkl'
#pickle.dump(XGB, open(filename, 'wb'))

# Later, you can load the saved model using:
#loaded_model = pickle.load(open(filename, 'rb'))
